The ESC403 cluster comes with [Catherine Devlin's `%sql`-magic for IPython][1]; this allows you to run SQL queries from the IPython notebook, and intermix them with Python code.

[1]: https://github.com/catherinedevlin/ipython-sql

Before we can use the `%sql` syntax, two steps must be taken:

* Load the IPython-SQL bridge code

In [1]:
%load_ext sql


* Connect to an actual database; this must be the first `%sql` statement (the funny `rmurri@/lustre` syntax is correct and means "connect to the PostGreSQL DB named `lustre` running on *this* host as user `rmurri`"  -- please replace `rmurri` with your local user name):

In [2]:
%sql postgresql://muesim@/lustre


u'Connected: muesim@lustre'

Now you can run 1-line SQL queries by prefixing them with `%sql`:

In [3]:
%sql select * from lustre limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path
usr356,i5105,1360399005,1368700859,4,2248,/scratch/bioc/usr356/param/ZINC04992888.rtf
usr356,i5105,1360387170,1368792474,8,6640,/scratch/bioc/usr356/param/ZINC71832434.prm
usr356,i5105,1360416215,1368785748,8,4480,/scratch/bioc/usr356/param/ZINC71850771.prm
usr356,i5105,1360404430,1368792486,4,2556,/scratch/bioc/usr356/param/ZINC34729086.rtf
usr356,i5105,1360420861,1368774402,4,2065,/scratch/bioc/usr356/param/ZINC45891658.rtf


It is also possible to run multi-line (or multiple) SQL queries by using the `%%sql` syntax instead.  Note that in this case the SQL instructions *must not* be on the same line as the `%%sql` magic marker:

In [ ]:
%%sql
select count(*) from lustre;
select distinct count(usr) from lustre;
select distinct count(grp) from lustre;

The `%%time` magic prints the time taken to evaluate a cell (which comes handy when doing performance comparisons):

In [ ]:
%%time

import time
time.sleep(5)

----

**Note:** to keep running times low, we will be using table `lustre_sample` throughout, which contains a sample of 5% the rows of the original `lustre` table.

## 1. Is it possible to convert fields atime and mtime to PostgreSQL's TIMESTAMP type?

Yes, it *is* possible to alter a SQL table definition after the table has been created.  Look at the documentation for the [ALTER TABLE](https://www.tutorialspoint.com/sql/sql-alter-command.htm) statement.

We shall break this down into steps: (1) create a new table, (2) populate it, then (3) alter the definition and (4) fill the new column with values.

In [5]:
%sql drop table times
%sql create table times as (select * from lustre_sample);

Done.
1519053 rows affected.


[]

In [6]:
%sql alter table times add column atime_timestamp timestamp;

Done.


[]

In [7]:
%sql update times set atime_timestamp=to_timestamp(atime);

1519053 rows affected.


[]

Show some data from the table we created::

In [8]:
%sql select atime, atime_timestamp, path from times limit 3;

3 rows affected.


atime,atime_timestamp,path
1382131058,2013-10-18 21:17:38,/scratch/bioc/usr25/fegs/b3s/fegscoor/fegs78/coor56.xtc
1374746779,2013-07-25 10:06:19,/scratch/bioc/usr25/mddock/y3ma_059/t1.sh
1366377914,2013-04-19 13:25:14,/scratch/bioc/usr25/soft/SuiteSparse/CAMD/Doc/docdiff


## 2. Can you count the number of files in a given directory?

Yes, using SQL's `like` string matching operator, which allows any
part of a string to be matched by the `%` character (i.e., just like
`*` for file names)::

In [9]:
%sql select count(path) from lustre_sample where path like '/scratch/bioc/usr384/%'

1 rows affected.


count
129


## 3. Can you find the directory that holds the largest number of files? 

In [ ]:
This is possible. One way would be to split the path string into "path" and "filename", but there is no standard way to do this
string splitting. A database will generally have a way to do this, but it will be specific to that database.

## 4. Can you find the directory tree that holds the largest number of files?

It's possible, but it becomes very complicated. This is trivial to do in the map/reduce framework. The problem is that now the string splitting does not work anymore, since we will have a hierarchy of paths. SQL in general is not well equipped to deal with hierarchies.